In [1]:
import duckdb
import threading
import random
import time
import datetime
import gradio as gr
import plotly.graph_objects as go

loop_count = 0
should_continue = True

output_text = "Initializing..."

In [2]:
def setup_database():
    conn = duckdb.connect(database='iot_data.db')
    conn.execute("""
        DROP TABLE IF EXISTS raw_data;
        DROP VIEW IF EXISTS summary;
    """)
    conn.execute("""
        CREATE TABLE IF NOT EXISTS raw_data (
            device_id INTEGER,
            temperature REAL,
            timestamp TIMESTAMP
        )
    """)
    conn.execute("""
        CREATE VIEW summary AS 
            SELECT 
                device_id as device,
                AVG(temperature) as avg_temp,
                DATE_TRUNC('day', timestamp) as date
            FROM 
                raw_data
            GROUP BY
                device_id, DATE_TRUNC('day', timestamp)
    """)

    conn.close()

In [3]:
# Thread for data ingestion
def data_ingestion_thread():
    global should_continue
    conn = duckdb.connect(database='iot_data.db')
    while should_continue:
        # Simulate data ingestion from IoT devices
        device_id = random.randint(1, 35)
        temperature = random.uniform(16, 42)
        random_timestamp = get_random_date()

        conn.begin()
        conn.execute("INSERT INTO raw_data (device_id, temperature, timestamp) VALUES (?, ?, ?)", (device_id, temperature, random_timestamp))
        conn.commit()
        time.sleep(1)
    conn.close()

In [4]:
def query_data_count():
    global loop_count, should_continue, output_text
    conn = duckdb.connect(database='iot_data.db')
    while loop_count < 2000 and should_continue:
        result = conn.execute("SELECT distinct device, avg_temp ,date FROM summary order by device,date").fetchall()
        output_text = format_results(result)
        loop_count += 1
        time.sleep(1)
    conn.close()

    output_text = output_text + "\nProcessing Complete"
    should_continue = False

def get_random_date():
    today = datetime.datetime.now()
    days_back = 180  
    random_days = random.randint(0, days_back)  # Generates a random number of days
    random_date = today - datetime.timedelta(days=random_days)
    return random_date

# Formats our output data
def format_results(results):
    output_text = f"Summary Data\n{'Device ID':<23}{'AVG Temp':<23}{'Date'}\n"
    for row in results:
        device, avg_temp, ts = row
        date_str = ts.strftime("%Y-%m-%d") if ts is not None else "N/A"
        output_text += f"{device:<30}{avg_temp:<30.2f}{date_str}\n"
    return output_text

In [5]:
# Returns the current value of output_text
def get_output_text():
    global output_text
    return output_text

# Gradio dashboard with Plotly integration
def update_dashboard():
    conn = duckdb.connect(database='iot_data.db')
    result = conn.execute("SELECT device, AVG(avg_temp) as avg_temp FROM summary GROUP BY device ORDER BY device").fetchall()
    conn.close()
    devices = [row[0] for row in result]
    avg_temps = [row[1] for row in result]

    fig = go.Figure(data=[
        go.Bar(x=devices, y=avg_temps, text=avg_temps, textposition='auto', marker=dict(color='rgba(0, 123, 255, 0.6)'))
    ])
    fig.update_layout(
        title='Average Temperature by Device',
        xaxis_title='Device ID',
        yaxis_title='Average Temperature (°C)',
        template='plotly_white'
    )
    return fig

def setup_gradio():
    with gr.Blocks() as demo:
        gr.Markdown("""
        # IoT Data Monitoring Dashboard
        This dashboard provides a live view of IoT device data, including average temperatures per device.
        """)
        output_box = gr.Textbox(label="Output Text", lines=10)
        graph = gr.Plot(update_dashboard)

        # refresh every 3 seconds
        def update_output_text():
            while True:
                time.sleep(3)
                yield get_output_text()

        def update_graph():
            while True:
                time.sleep(3)
                yield update_dashboard()

        demo.load(update_output_text, None, output_box)
        demo.load(update_graph, None, graph)

    return demo

In [6]:
if __name__ == "__main__":
    setup_database()

    # Start threads
    ingestion_thread = threading.Thread(target=data_ingestion_thread, daemon=True)
    count_thread = threading.Thread(target=query_data_count, daemon=True)

    ingestion_thread.start()
    count_thread.start()

    # Launch the Gradio app
    app = setup_gradio()
    app.launch(share=False, server_port=7860, height=1080, width=1920)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
